## Intervalos de confianza
Msc Renzo Claure

In [ ]:
import pandas as pd
import numpy as np
import math
from scipy.stats import norm

In [ ]:
galton = pd.read_csv('galton.csv')
print(galton.shape)
galton.head()

In [ ]:
#Intervalo de confianza de la estatura del papá
import numpy as np
from scipy.stats import norm

# Datos de muestra
padre = galton['father']

# Nivel de confianza (90%, 95%, 99%)
nivel_confianza = [0.9, 0.95, 0.99]

# Calcular la media y desviación estándar de los datos de muestra
sample_mean = np.mean(padre)
sample_std = np.std(padre, ddof=1)

# Calcular el tamaño de la muestra
sample_size = len(padre)



In [ ]:
print('media_muestra:' , sample_mean)
print('desv_est:' , sample_std)
print('n_obs:' , sample_size)

In [ ]:
norm.ppf((1 + 0.99) / 2)

In [ ]:
# Calcular el valor crítico de la distribución normal estándar para cada nivel de confianza
critical_values = [norm.ppf((1 + confidence) / 2) for confidence in nivel_confianza]

# Calcular el error estándar
standard_error = sample_std / np.sqrt(sample_size)

# Calcular los límites del intervalo de confianza para cada nivel
confidence_intervals = [
    (sample_mean - critical_value * standard_error, sample_mean + critical_value * standard_error)
    for critical_value in critical_values
]

# Imprimir los intervalos de confianza
for i, confidence_interval in enumerate(confidence_intervals):
    confidence_level = nivel_confianza[i]
    print(f"Intervalo de confianza al {confidence_level*100}%: {confidence_interval}")

In [ ]:
critical_values

In [ ]:
#intervalo de confianza para los votantes
import math
from scipy.stats import norm

n = 100  # Tamaño de la muestra
p = 56 / n  # Proporción observada

confidence_level = 0.95  # Nivel de confianza

# Calcular el error estándar utilizando la fórmula de proporción
standard_error = math.sqrt((p * (1 - p)) / n)

# Calcular el valor crítico de la distribución normal estándar para el nivel de confianza dado
z_critical = norm.ppf((1 + confidence_level) / 2)

# Calcular los límites del intervalo de confianza
lic = p - z_critical * standard_error
lsc = p + z_critical * standard_error

print(f"Intervalo de confianza del {confidence_level * 100}%: ({lic:.3f}, {lsc:.3f})")


In [ ]:
from statsmodels.stats.proportion import proportion_confint

exitos = 56
n = 100

conf_int = proportion_confint(exitos, n, method='wilson')

print("Intervalo de confianza:", conf_int)

## Cobertura

In [ ]:
#simulacion de intérvalo de confianza de Wald
#mientras más grande es n, mas cobertura se tiene
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import binom, norm

n = 20
pvals = np.arange(0.1, 0.9, 0.05)
nosim = 1000

coverage = []
for p in pvals:
    phats = binom.rvs(n=n, p=p, size=nosim) / n
    ll = phats - norm.ppf(0.975) * np.sqrt(phats * (1 - phats) / n)
    ul = phats + norm.ppf(0.975) * np.sqrt(phats * (1 - phats) / n)
    coverage.append(np.mean((ll < p) & (ul > p)))
   

# Graficar
plt.plot(pvals, coverage, marker='o')
plt.axhline(y=0.95, color='gray', linestyle='dashed')
plt.ylim(0.5, 1)
plt.xlabel('Proporción real')
plt.ylabel('Cobertura')
plt.title('Cobertura del intervalo de Wald')
plt.show()

#Por que tan poca cobertura
#Como afecta n
#Como corregirla  (+2/n+4)

In [ ]:
#Una compresora falló 5 veces en 94,32 días. Calcule el 95% de intervalo de confianza para el ratio de falla por día.

In [ ]:
import numpy as np
from scipy.stats import norm

# Datos del problema
x = 5
t = 94.32

# Calcular la tasa observada (lambda hat)
lambda_hat = x / t

# Nivel de confianza del 95%
confianza = 0.95
alpha = 1 - confianza

# Calcular el valor crítico Z_alpha/2 para la distribución normal estándar
valor_critico = norm.ppf(1 - alpha/2)

# Calcular los límites del intervalo de confianza
limite_inferior = lambda_hat - valor_critico * np.sqrt(lambda_hat / t)
limite_superior = lambda_hat + valor_critico * np.sqrt(lambda_hat / t)

# Redondear los resultados a 3 decimales
limite_inferior = round(limite_inferior, 3)
limite_superior = round(limite_superior, 3)

# Imprimir los resultados
print(f"Intervalo de confianza al {confianza * 100}% para la tasa de falla por día:")
print(f"Limite inferior: {limite_inferior}")
print(f"Limite superior: {limite_superior}")


### Intervalo de Poisson

In [ ]:
from scipy.stats import poisson
import numpy as np

# Datos del problema
x = 5
t = 94.32

# Calcular la tasa observada (lambda hat)
lambda_hat = x / t

# Realizar la prueba de Poison y obtener el intervalo de confianza
resultado_prueba = poisson.interval(0.95, x/t )

# Escalar el intervalo según la tasa observada
limite_inferior = lambda_hat * np.exp(resultado_prueba[0])
limite_superior = lambda_hat * np.exp(resultado_prueba[1])

# Imprimir los resultados
print(f"Intervalo de confianza al 95% para la tasa de falla por día:")
print(f"Limite inferior: {limite_inferior:.4f}")
print(f"Limite superior: {limite_superior:.4f}")


In [ ]:
resultado_prueba

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from scipy.stats import norm

lambdavals = np.arange(0.005, 0.1, 0.01)
nosim = 1000
t = 1000
coverage = np.zeros(len(lambdavals))

for i in range(len(lambdavals)):
    lambda_val = lambdavals[i]
    lhats = np.random.poisson(lam=lambda_val * t, size=nosim) / t
    ll = lhats - norm.ppf(0.975) * np.sqrt(lhats / t)
    ul = lhats + norm.ppf(0.975) * np.sqrt(lhats / t)
    coverage[i] = np.mean((ll < lambda_val) & (ul > lambda_val))

plt.plot(lambdavals, coverage, marker='o')
plt.xlabel('Lambda')
plt.ylabel('Cobertura')
plt.title('Cobertura del intervalo de confianza asintótico de poisson')
plt.grid(True)
plt.show()


In [ ]:
# Ejercicio Flores

In [ ]:
from sklearn import datasets

iris = datasets.load_iris()

X = iris.data  
y = iris.target  

In [ ]:
iris.feature_names

In [ ]:
S = X[:,0]

In [ ]:
S

In [ ]:
len(y)

In [ ]:
sample_mean = np.mean(S)

In [ ]:
sample_std = np.std(S, ddof=1)

In [ ]:
sample_size = len(S)

In [ ]:
print('media_muestra:' , sample_mean)
print('desv_est:' , sample_std)
print('n_obs:' , sample_size)

In [ ]:
# Nivel de confianza (90%, 95%, 99%)
nivel_confianza = [0.9, 0.95, 0.99]

# Calcular el valor crítico de la distribución normal estándar para cada nivel de confianza
critical_values = [norm.ppf((1 + confidence) / 2) for confidence in nivel_confianza]

# Calcular el error estándar
standard_error = sample_std / np.sqrt(sample_size)

# Calcular los límites del intervalo de confianza para cada nivel
confidence_intervals = [
    (sample_mean - critical_value * standard_error, sample_mean + critical_value * standard_error)
    for critical_value in critical_values
]

# Imprimir los intervalos de confianza
for i, confidence_interval in enumerate(confidence_intervals):
    confidence_level = nivel_confianza[i]
    print(f"Intervalo de confianza al {confidence_level*100}%: {confidence_interval}")